<a href="https://colab.research.google.com/github/ampig2020/Data-Science-Hands-on-Practice/blob/main/Finding_optimal_locations_of_new_stores_using_Decision_Optimization_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows you how Decision Optimization can help to prescribe decisions for a complex constrained problem using CPLEX Modeling for Python to help determine the optimal location for a new store. This notebook requires the Commercial Edition of CPLEX engines, which is included in the latest Python XS + DO environment in Watson Studio.

**Finding Optimal Locations for New Stores**

This notebook is an example of how Decision Optimization can help to prescribe decisions for a complex constrained problem.

When you finish this notebook, you'll have a foundational knowledge of Prescriptive Analytics.

This notebook requires the Commercial Edition of CPLEX engines, which is included in the Default Python 3.7 XS + DO in Watson Studio.

Table of contents:

Describe the business problem


How decision optimization (prescriptive analytics) can help

Use decision optimization

Step 1: Import the docplex package

Step 2: Model the data

Step 3: Prepare the data

Step 4: Set up the prescriptive model

Define the decision variables

Express the business constraints

Express the objective

Solve with the Decision Optimization solve service

Step 5: Investigate the solution and run an example analysis

Summary

---



**Describe the business problem**

A fictional Coffee Company plans to open N shops in the near future and needs to determine where they should be located knowing the following:

Most of the customers of this coffee brewer enjoy reading and borrowing books, so the goal is to locate those shops in such a way that all the city public libraries are within minimal walking distance.

We use Chicago open data for this example.

We implement a K-Median model to get the optimal location of our future shops.

**How decision optimization can help**

Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes. It takes into account specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control of business outcomes.

Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.

Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.


With prescriptive analytics, you can:

Automate the complex decisions and trade-offs to better manage your limited resources.
Take advantage of a future opportunity or mitigate a future risk.
Proactively update recommendations based on changing events.
Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.

***Use decision optimization***

Step 1: Import the docplex package
